# Type et méthodes

*Julia* possède un système de type et de méthode qui lui confère une approche objet.
La fonction `typeof()` renvoie le type d'une variable de base `Int32`, `Float64`... *Julia* est conçu pour permettre facilement d'étendre l'environnement à de nouveau type de variable.

Le types sont organisés suivant un hiérarchie comme on peut le voir sur l'arborescence partielle ci-dessous


In [3]:
using AbstractTrees

AbstractTrees.children(x::Type) = subtypes(x)

print_tree(AbstractArray)

AbstractArray
├─ AbstractRange
│  ├─ LinRange
│  ├─ OrdinalRange
│  │  ├─ AbstractUnitRange
│  │  │  ├─ IdentityUnitRange
│  │  │  ├─ OneTo
│  │  │  ├─ Slice
│  │  │  └─ UnitRange
│  │  └─ StepRange
│  └─ StepRangeLen
├─ AbstractSlices
│  └─ Slices
├─ ExceptionStack
├─ LogRange
├─ LogicalIndex
├─ MethodList
├─ ReinterpretArray
├─ ReshapedArray
├─ SCartesianIndices2
├─ WithoutMissingVector
├─ BitArray
├─ CartesianIndices
├─ AbstractRange
├─ LinRange
│  ├─ OrdinalRange
│  │  ├─ AbstractUnitRange
│  │  │  ├─ IdentityUnitRange
│  │  │  ├─ OneTo
│  │  │  ├─ Slice
│  │  │  ├─ StmtRange
│  │  │  └─ UnitRange
│  │  └─ StepRange
│  └─ StepRangeLen
├─ BitArray
├─ ExceptionStack
├─ LinearIndices
├─ LogRange
├─ MethodList
├─ TwoPhaseDefUseMap
├─ TwoPhaseVectorView
├─ DenseArray
│  ├─ Array
│  ├─ CodeUnits
│  ├─ Const
│  ├─ GenericMemory
│  └─ UnsafeView
├─ AbstractTriangular
│  ├─ LowerTriangular
│  ├─ UnitLowerTriangular
│  ├─ UnitUpperTriangular
│  └─ UpperTriangular
├─ Adjoint
├─ Bidiagonal
├─ 

Dans cette arborescence, certains types sont "abstraits" et d'autres "concrets".

In [4]:
isconcretetype(Rational{Int32}), isconcretetype(Float64)

(true, true)

Un réel sera forcemment de type concret `Float64` ou `Float32` par exemple, et pourra être utilisé comme argument par toutes les fonctions acceptant le type abstrait `AbstractFloat`

In [5]:
Int32 <: Complex

false

# Méthodes

A chaque fonction est associée une méthode dépendante du type d'entrée comme dans ce qui suit suivant que l'entrée soit un entier ou pas.

In [6]:
function f(x::Any) 
    sin(x+1)
end

f (generic function with 1 method)

In [7]:
function f(n::Integer)
    n
end

f (generic function with 2 methods)

In [8]:
f(3.0)

-0.7568024953079282

In [9]:
f(3)

3

In [10]:
methods(f)

# 2 methods for generic function "f" from Main:
 [1] f(n::Integer)
     @ In[7]:1
 [2] f(x)
     @ In[6]:1

In [11]:
f(3)

3

In [12]:
f(im)

1.2984575814159773 + 0.6349639147847361im

In [13]:
f(-2)

-2

In [14]:
+

+ (generic function with 198 methods)

In [15]:
*(3,2)

6

In [16]:
f(sqrt(2))

0.6649143126867011

# Construction d'un nouveau Type de variable

En premier lieu il faut définir un type abstrait puis une instance sous-hiérarchiquement concrète :

In [17]:
struct Grid1 
    debut::Float64
    fin::Float64
    n::Int32
end

In [18]:
@show a = Grid1(0, 1, 2)

a = Grid1(0, 1, 2) = Grid1(0.0, 1.0, 2)


Grid1(0.0, 1.0, 2)

In [19]:
a.debut

0.0

In [20]:
a.fin

1.0

In [21]:
a.n

2

In [22]:
a.n = 10

LoadError: setfield!: immutable struct of type Grid1 cannot be changed

In [23]:
a

Grid1(0.0, 1.0, 2)

# Surcharge des opérateurs

La surcharge des opérations usuelles se fait en définissant une nouvelle méthode associé au nouveau type pour chaque opérateur, commençons par surcharger l'affichage à l'écran de notre nouveau type. Pour cela on va ajouter une méthode à la fonction "show"

In [24]:
function Base.show(io::IO,g::Grid1)
    print(io, "Grid 1d : début $(g.debut) , fin $(g.fin) , $(g.n) éléments\n")
end

In [25]:
Base.show(a)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [26]:
println(a)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments



In [27]:
a

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [28]:
@show a

a = Grid 1d : début 0.0 , fin 1.0 , 2 éléments



Grid 1d : début 0.0 , fin 1.0 , 2 éléments


## Addition, soustraction ...

Ces fonctions sont de la forme +(), -() c'est à dire

In [29]:
import Base:+

function +(g::Grid1, n::Int)
    g.n += n
    return g
end

+ (generic function with 199 methods)

In [30]:
a = Grid1(0,1,2)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [31]:
a+2

LoadError: setfield!: immutable struct of type Grid1 cannot be changed

In [32]:
a += 1

LoadError: setfield!: immutable struct of type Grid1 cannot be changed

Attention l'addition n'est pas forcément commutative !

In [33]:
try 
    2 + a
catch e
    showerror(stdout, e)
end

MethodError: no method matching +(::Int64, ::Grid1)
The function `+` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  +(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:596
  +(::Real, ::Complex{Bool})
   @ Base complex.jl:322
  +(::Integer, ::AbstractChar)
   @ Base char.jl:247
  ...


ni unaire !

In [34]:
try 
    -a
catch e
    showerror(stdout, e)
end 

MethodError: no method matching -(::Grid1)
The function `-` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  -(::Bool, ::Complex{Bool})
   @ Base complex.jl:310
  -(::Bool, ::Bool)
   @ Base bool.jl:167
  -(::Bool)
   @ Base bool.jl:164
  ...


Notez le message d'erreur qui est très claire !

In [35]:
try
    a+[1,2]
catch e
    showerror(stdout, e)
end 

MethodError: no method matching +(::Grid1, ::Vector{Int64})
The function `+` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  +(::Any, ::Any, ::Any, ::Any...)
   @ Base operators.jl:596
  +(::Grid1, ::Int64)
   @ Main In[29]:3
  +(::Array, ::Array...)
   @ Base arraymath.jl:12
  ...


## Autres surcharges

Toutes les fonctions usuelles sont surchargeable sans limite : size(); det() ...

In [36]:
function Base.size(g::AbstractGrid)
    return g.n
end

LoadError: UndefVarError: `AbstractGrid` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [37]:
size(a)

LoadError: MethodError: no method matching size(::Grid1)
The function `size` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  size([91m::ZMQ.Message[39m)
[0m[90m   @[39m [35mZMQ[39m [90m~/.julia/packages/ZMQ/Lz9O7/src/[39m[90m[4mmessage.jl:164[24m[39m
[0m  size([91m::Base.MethodList[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mreflection.jl:1193[24m[39m
[0m  size([91m::Base.ExceptionStack[39m)
[0m[90m   @[39m [90mBase[39m [90m[4merrorshow.jl:1066[24m[39m
[0m  ...


In [38]:
using LinearAlgebra

function LinearAlgebra.det(g::Grid1)
    g.fin-g.debut
end 

In [39]:
det(a)

1.0

# Type et constructeurs

Chaque langage "objet" définit un constructeur pour ces objets. Nous avons déjà utilisé un constructeur générique qui rempli chaque champ du nouveau type. Il est possible de faire une variante suivant le nombre d'arguments d'entrée et de leur type 

In [40]:
abstract type AbstractGrid end # juste en dessous de Any

struct Grid1D <: AbstractGrid
    debut::Float64
    fin::Float64
    n::Int32
    
    # constructeurs par défaut sans argument
    function Grid1D()
        new(0,0,0)
    end
    
    # constructeurs par défaut avec argument
    function Grid1D(a,b,c)
        @assert c > 0 "Attention la taille de la grille est négative ou nulle"
        new(a,b,c)
    end
end

In [41]:
b = Grid1D(0, 1, -1)

LoadError: AssertionError: Attention la taille de la grille est négative ou nulle

Il devient possible de déterminer un constructeurs pour différentes entrées.

Il faut au préalable bien penser sa hiérarchie de type et écrire autant de fonctions constructeurs que de cas d'initialisation du nouveau type.

# Les Itérateurs

Il est possible sur un type nouveau de définir un itérateur, comme ici de parcourir les points de la grille, définissons (surchargeons) de nouvelles fonctions ou plutôt méthodes : 

In [42]:
Base.iterate(g::Grid1D, state=g.debut) = begin
   state > g.fin ? nothing : (state, state+(g.fin-g.debut)/g.n)
end

In [43]:
grid = Grid1D(0, 2 , 10)

Grid1D(0.0, 2.0, 10)

In [44]:
for x in grid
    println(x)
end

0.0
0.2
0.4
0.6000000000000001
0.8
1.0
1.2
1.4
1.5999999999999999
1.7999999999999998
1.9999999999999998


Il devient possible de construire des itérateurs sur une grille 2d, 3d renvoyant les coordonnées des points de la grille... Mais on peut imaginer sur une triangulation etc... 